# Proscenium Legal Demo (GraphRAG)

## Prerequisites

1. Obtain a `TOGETHER_API_KEY` from https://together.ai/ and store it as an environment variable or Colab secret.
2. Obtain a Neo4j Sandbox https://neo4j.com/sandbox/



## Setup

In [ ]:
!git clone https://github.com/The-AI-Alliance/proscenium.git

In [ ]:
%cd proscenium

In [ ]:
!python -m pip install .

In [ ]:
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
from rich import print
from rich.panel import Panel
from rich.prompt import Prompt
import asyncio

import nest_asyncio
nest_asyncio.apply()

## Configure

The folowing import defines

1. Hugging Face dataset id (NH caselaw)
2. Filenames (entity csv)
3. Model ids (for extraction, embedding, and generation)
4. Domain-specific logic for processing caselaw and user queries


In [ ]:
import demo.domains.legal as legal_config

print(legal_config.default_chunk_extraction_model_id)

Now set secrets for accessing Together and Neo4j

In [ ]:
from google.colab import userdata

api_key = userdata.get('TOGETHER_API_KEY')
os.environ['TOGETHER_API_KEY'] = api_key

neo4j_uri = userdata.get('NEO4J_URI')
neo4j_username = userdata.get("NEO4J_USERNAME")
neo4j_password = userdata.get("NEO4J_PASSWORD")


## Extract Knowledge Graph

In [ ]:
from proscenium.scripts.graph_rag import extract_entities

extract_entities(
  legal_config.retrieve_documents,
  legal_config.doc_as_rich,
  legal_config.entity_csv_file,
  legal_config.doc_direct_triples,
  legal_config.default_chunk_extraction_model_id,
  legal_config.triples_from_chunk)


## Load Knowledge Graph

In [ ]:
from proscenium.verbs.know import knowledge_graph_client

driver = knowledge_graph_client(neo4j_uri, neo4j_username, neo4j_password)

In [ ]:
from proscenium.scripts.graph_rag import load_entity_graph

load_entity_graph(
    driver,
    legal_config.entity_csv_file,
    legal_config.add_triple)

driver.close()

## Show Knowledge Graph

In [ ]:
from proscenium.scripts.graph_rag import show_entity_graph

driver = knowledge_graph_client(neo4j_uri, neo4j_username, neo4j_password)

show_entity_graph(driver)

driver.close()

## Load Entity Resolver

In [ ]:
from proscenium.verbs.vector_database import embedding_function

embedding_fn = embedding_function(legal_config.embedding_model_id)
print("Embedding model", legal_config.embedding_model_id)

from proscenium.verbs.vector_database import create_vector_db

vector_db_client = create_vector_db(legal_config.milvus_uri, embedding_fn, overwrite=True)
print("Vector db stored at", legal_config.milvus_uri)

driver = knowledge_graph_client(neo4j_uri, neo4j_username, neo4j_password)

from proscenium.verbs.vector_database import collection_name
from proscenium.scripts.graph_rag import load_entity_resolver
load_entity_resolver(driver, vector_db_client, embedding_fn, collection_name)

driver.close()
vector_db_client.close()

## Answer Question

In [ ]:
question = "How has Judge Kenison used Ballou v. Ballou to rule on cases?"

In [ ]:
driver = knowledge_graph_client(neo4j_uri, neo4j_username, neo4j_password)

from proscenium.verbs.vector_database import vector_db
vector_db_client = vector_db(legal_config.milvus_uri)

from proscenium.scripts.graph_rag import answer_question

answer = answer_question(
    question,
    legal_config.default_query_extraction_model_id,
    vector_db_client,
    legal_config.embedding_model_id,
    driver,
    legal_config.default_generation_model_id,
    legal_config.triples_from_query,
    legal_config.generation_prompts,
    )

if answer:
    print(Panel(answer, title="Answer"))
else:
    print("No answer")

driver.close()
vector_db_client.close()
